# Proyecto de Sistema de clasificación de imágenes

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.preprocessing import image

2025-05-13 02:08:17.082177: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 02:08:17.308091: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-13 02:08:17.405507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747102097.533448    2424 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747102097.567620    2424 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747102097.818658    2424 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [5]:
ruta_datos = "../data/train"

carpeta_perros = sistema_operativo.path.join(ruta_datos, "dog")

archivos_imagenes_perro = sistema_operativo.listdir(carpeta_perros)

imagenes_perro = []
for indice in range(9):
    ruta_imagen = sistema_operativo.path.join(carpeta_perros, archivos_imagenes_perro[indice])
    imagen_cargada = img.load_img(ruta_imagen)
    arreglo_imagen = img.img_to_array(imagen_cargada) / 255.0
    imagenes_perro.append(arreglo_imagen)

plt.figure(figsize=(12, 8))

for indice in range(9):
    plt.subplot(3, 3, indice + 1)
    plt.imshow(imagenes_perro[indice])
    plt.title('Dog Image')
    plt.axis('off')

plt.tight_layout()
plt.show()


NameError: name 'sistema_operativo' is not defined